In [50]:

import pandas as pd
import numpy as np
import re
from datetime import datetime


### EDA PROFILE:

In [51]:
profile = pd.read_csv('../Clean Files/Profile/0V-profile_killers.csv')

In [52]:
#IDENTIFYING NULLS AND DATA TYPE

profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              711 non-null    object 
 1   Nicknames         469 non-null    object 
 2   Age               243 non-null    float64
 3   Born Date         444 non-null    object 
 4   Born Location     420 non-null    object 
 5   Died Date         308 non-null    object 
 6   Died Location     227 non-null    object 
 7   Conviction        502 non-null    object 
 8   Criminal Penalty  577 non-null    object 
 9   Date Apprehended  543 non-null    object 
dtypes: float64(1), object(9)
memory usage: 55.7+ KB


In [53]:
# SPLITING BORN LOCATION, TO CREATE NEW COLUMN WITH COUNTRY


profile['born country'] = profile['Born Location'].apply(
    lambda x: x.split(',')[-1].strip() if pd.notnull(x) else None)

In [54]:
# CAN IDENTIFY DATA AGE IS ACTUALY AGE THE PERSON DIED

#pd.set_option('display.max_rows', None)
profile[profile['Age'].notna()].head()


,Name,Nicknames,Age,Born Date,Born Location,Died Date,Died Location,Conviction,Criminal Penalty,Date Apprehended,born country
0,Luis Garavito,"The Beast, Goofy, Conflict, Bonifacio Morera L...",66.0,1957-01-25,"Gnova, Quindo, Colombia",2023-10-12,"Valledupar, Colombia",NaN,NaN,1999-04-22,Colombia
4,Daniel Camargo Barbosa,"The Sadist of El Charquito, The Monster of the...",64.0,1930-01-22,"Anolaima, Cundinamarca, Colombia",1994-11-13,"Guayaquil, Ecuador",NaN,"25 years imprisonment, 16 years imprisonment",1974-05-03,Colombia
5,Pedro Rodrigues Filho,"Killer Petey, Pedrinho Matador",68.0,1954-10-29,"Santa Rita do Sapuca, Minas Gerais, Brazil",2023-03-05,"Mogi das Cruzes, So Paulo, Brazil",NaN,"30 years imprisonment (1973 to 2007), 8 years ...",NaN,Brazil
10,Samuel Little,"Samuel Mc, Dowell, The Choke--Stroke Killer, M...",80.0,1940-06-07,NaN,2020-12-30,NaN,Murder,Four life sentence(s) without the possibility ...,2012-09-05,None
13,Andrei Chikatilo,"The Butcher of Rostov, The Forest Strip Killer...",57.0,1936-10-16,", Sumy Oblast, Ukrainian SSR, Soviet Union",1994-02-14,"Novocherkassk, Rostov Oblast, Russia","Murder With Aggravating Circumstances, Seual A...",death,1990-11-20,Soviet Union


In [55]:
# ENSURING COLUMNS ARE IN DATETIME FORMAT, THEN CREATING A COLUNM APPREHENDED AGE AS INT

profile['Date Apprehended'] = pd.to_datetime(profile['Date Apprehended'], errors='coerce')
profile['Born Date'] = pd.to_datetime(profile['Born Date'], errors='coerce')

profile['Apprehended Age'] = ((profile['Date Apprehended'] - profile['Born Date']).dt.days // 365).astype('Int64')


In [56]:
profile['born country'].unique()

array(['Colombia', 'Pakistan', 'USSR', 'Brazil', 'India', 'Iran', 'China',
       'Abul DjabarAfghanistan', None, 'Protectorate of Uganda',
       'Soviet Union', 'Indonesia', 'South Africa', 'Byelorussian SSR',
       'Morocco', 'Romania', 'Mexico', 'Russian Empire', 'Egypt',
       'Greece', 'Maoupa Cedric MaakeSouth Africa', 'Canada', 'England',
       'German Empire', 'Hungary', 'France', 'Ecuador', 'RSFSR',
       'South Korea', 'Turkey', 'United States', 'Martinique', 'Peru',
       'Ukrainian SSR', 'Italy', 'Imperial State of Iran', 'Spain',
       'Elias XitavhudziUnion of South Africa', 'Uttar Pradesh',
       'Argentina', 'Union of South Africa', 'Norway', 'Poland', 'Chile',
       'Germany', 'French Tunisia', 'United States of Brazil',
       'Venezuela', 'Tajik SSR', 'Latvia',
       'Johannes MashianeSouth Africa', 'Georgian SSR',
       'North German Confederation', 'Scotland', 'Yemen',
       'occupied Austria', 'Belgium', 'Japan', 'So Paulo', 'Latvian SSR',
       'UkSS

In [57]:
to_continent = {
    'Colombia': 'South America',
    'Brazil': 'South America',
    'Argentina': 'South America',
    'Peru': 'South America',
    'Mexico': 'North America',
    'Canada': 'North America',
    'United States': 'North America',
    'US': 'North America',
    'Texas': 'North America',
    'New York': 'North America',
    'Massachusetts': 'North America',
    'England': 'Europe',
    'Scotland': 'Europe',
    'Wales': 'Europe',
    'United Kingdom': 'Europe',
    'UK': 'Europe',
    'France': 'Europe',
    'Germany': 'Europe',
    'Italy': 'Europe',
    'Poland': 'Europe',
    'Hungary': 'Europe',
    'Romania': 'Europe',
    'Russia': 'Europe',
    'USSR': 'Europe',
    'Soviet Union': 'Europe',
    'Byelorussian SSR': 'Europe',
    'Ukrainian SSR': 'Europe',
    'Tajik SSR': 'Asia',
    'Uzbek SSR': 'Asia',
    'Moldavian SSR': 'Europe',
    'Latvia': 'Europe',
    'Latvian SSR': 'Europe',
    'Georgian SSR': 'Europe',
    'RSFSR': 'Europe',
    'Imperial Germany': 'Europe',
    'German Empire': 'Europe',
    'German Confederation': 'Europe',
    'West Germany': 'Europe',
    'East Germany': 'Europe',
    'Austria': 'Europe',
    'occupied Austria': 'Europe',
    'Czechoslovakia': 'Europe',
    'Czech Republic': 'Europe',
    'Slovakia': 'Europe',
    'Portugal': 'Europe',
    'Belgium': 'Europe',
    'Netherlands': 'Europe',
    'Sweden': 'Europe',
    'Denmark': 'Europe',
    'Norway': 'Europe',
    'Ireland': 'Europe',
    'Switzerland': 'Europe',
    'Yugoslavia': 'Europe',
    'SFR Yugoslavia': 'Europe',
    'Socialist Federal Republic of Yugoslavia': 'Europe',
    'Kingdom of Yugoslavia': 'Europe',
    'Kingdom of Romania': 'Europe',
    'Israel': 'Asia',
    'Iran': 'Asia',
    'Imperial State of Iran': 'Asia',
    'India': 'Asia',
    'Pakistan': 'Asia',
    'Indonesia': 'Asia',
    'Malaysia': 'Asia',
    'China': 'Asia',
    'Singapore': 'Asia',
    'Japan': 'Asia',
    'Empire of Japan': 'Asia',
    'Occupied Japan': 'Asia',
    'FukuokaOccupied Japan': 'Asia',
    'TaeiIraq': 'Asia',
    'Turkey': 'Asia',
    'centuryOttoman Empire': 'Asia',
    'Afghanistan': 'Asia',
    'Abul DjabarAfghanistan': 'Asia',
    'Zhejiang': 'Asia',
    'Chongqing': 'Asia',
    'Egypt': 'Africa',
    'Morocco': 'Africa',
    'South Africa': 'Africa',
    'Maoupa Cedric MaakeSouth Africa': 'Africa',
    'Johannes MashianeSouth Africa': 'Africa',
    'Union of South Africa': 'Africa',
    'Elias XitavhudziUnion of South Africa': 'Africa',
    'Protectorate of Uganda': 'Africa',
    'Kenya': 'Africa',
    'Nigeria': 'Africa',
    'French Tunisia': 'Africa',
    'Cape Colony': 'Africa',
    'Australia': 'Oceania',
    'Martinique': 'North America',
    'Costa Rica': 'North America',
    'Guatemala': 'North America',
    'Venezuela': 'South America',
    'So Paulo': 'South America',
    'Rio de Janeiro': 'South America',
    'Mxico': 'North America',
    'sMexico': 'North America',
    'Gabriel Garza HothMexico': 'North America',
    'Hong Kong': 'Asia',
    'British Hong Kong': 'Asia',
    'Israel': 'Asia',
    's Republic': None,
    's': None,
    'None': None,
    'unknown': 'Unknown',
    'Territory of Hawaii': 'Oceania',
    'Ontario Canada': 'North America',
    'Grand Duchy of Hesse': 'Europe',
    'Kingdom of Prussia': 'Europe',
    'Westphalia': 'Europe',
    'North German Confederation': 'Europe',
    'Imperial Germany': 'Europe',
    'occupied Germany': 'Europe',
    'Congress Poland': 'Europe',
    'BSSR': 'Europe'
}

profile['Continent'] = profile['born country'].map(to_continent)


In [58]:
profile[profile['Apprehended Age']<20]

,Name,Nicknames,Age,Born Date,Born Location,Died Date,Died Location,Conviction,Criminal Penalty,Date Apprehended,born country,Apprehended Age,Continent
42,Bruce George Peter Lee,NaN,NaN,1960-07-31,"Manchester, England",NaN,NaN,"Manslaughter, Arson",Indefinite hospital detention,1979-12-04,England,19,Europe
50,Juan Fernando Hermosa,Niño del Terror-Child of Terror,20.0,1976-02-28,"Clemente Baquerizo, Los Ros Province, Ecuador",1996-02-28,"Nueva Loja, Sucumbos Province, Ecuador",Murder,4 years imprisonment,1992-01-09,Ecuador,15,NaN
134,Denis Pischikov,"The Shivering Creature, Urals Raskolnikov",NaN,1983-04-12,"Bulanash, Sverdlovsk Oblast, RSFSR",NaN,NaN,Murder,life imprisonment,2003-04-05,RSFSR,19,Europe
212,Edmund Kemper,"Co-ed Killer, Co-ed Butcher, Ogre of Aptos The...",NaN,1948-12-18,NaN,NaN,NaN,First-Degree Murder ×,NaN,1964-08-27,None,15,NaN
389,Robert Nixon,The Brick Moron,19.0,1919-07-19,NaN,1939-06-16,NaN,Murder,death,1938-05-28,None,18,NaN
448,Craig Price,NaN,NaN,1973-10-11,NaN,NaN,NaN,NaN,NaN,1989-09-05,None,15,NaN
450,Cayetano Santos Godino,NaN,48.0,1896-10-31,"Buenos Aires, Argentina",1944-11-15,"Ushuaia, Argentina",NaN,NaN,1912-12-04,Argentina,16,South America
455,Peter Woodcock,NaN,71.0,1939-03-05,"Peterborough, Ontario, Canada",2010-03-05,"Penetanguishene, Ontario, Canada",NaN,NaN,1957-01-21,Canada,17,North America
459,Jürgen Bartsch,The Carnival Killer (Der Kirmesmörder),29.0,1946-11-06,occupied Germany,1976-04-28,"Eickelborn, West Germany",Murder,"life imprisonment, commuted 10 years of juveni...",1966-01-01,occupied Germany,19,Europe
476,Harvey Miguel Robinson,NaN,NaN,1974-12-06,NaN,NaN,NaN,"First Degree Murder Attempted Murder, Burglary",death,1993-12-06,None,19,NaN


In [59]:
def get_zodiac_sign(born):
    if pd.isnull(born):
        return None
    try:
       
        if isinstance(born, str):
            born = pd.to_datetime(born, errors='coerce')
        if pd.isnull(born):
            return None

        month = born.month
        day = born.day

        if (month == 3 and day >= 21) or (month == 4 and day <= 19):
            return 'Aries'
        elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
            return 'Taurus'
        elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
            return 'Gemini'
        elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
            return 'Cancer'
        elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
            return 'Leo'
        elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
            return 'Virgo'
        elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
            return 'Libra'
        elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
            return 'Scorpio'
        elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
            return 'Sagittarius'
        elif (month == 12 and day >= 22) or (month == 1 and day <= 19):
            return 'Capricorn'
        elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
            return 'Aquarius'
        elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
            return 'Pisces'
    except:
        return None

#CALLING THE FUNCTION AND CREATING A NEW COLUMN

profile['Zodiac Sign'] = profile['Born Date'].apply(get_zodiac_sign)


In [60]:
profile['Zodiac Sign'].value_counts()

Zodiac Sign
Sagittarius    46
Aquarius       44
Pisces         43
Capricorn      40
Libra          38
Scorpio        37
Aries          36
Virgo          36
Gemini         34
Leo            33
Cancer         30
Taurus         27
Name: count, dtype: int64

In [61]:
profile['Criminal Penalty'].unique()

array([nan,
       '7 years imprisonment in Colombia, extended to 9 due to in-prison homicide (1969 to 1978), 16 years imprisonment in Ecuador, released after 14 years (1980 to 1994), committed to mental hospital in Colombia (1994 to 1998)',
       'death', 'life imprisonment',
       '25 years imprisonment, 16 years imprisonment',
       '30 years imprisonment (1973 to 2007), 8 years imprisonment (7 served on good behavior, 2011 to 2018)',
       'Four life sentence(s) without the possibility of parole',
       'death, commuted to life imprisonment',
       '113 years imprisonment, released after 27 years',
       '49 life sentence(s) without the possibility of parole',
       '2,410 years imprisonment', '60 years imprisonment',
       'Execution by hanging', '580 years imprisonment',
       '34 years in jail',
       '14 years in prison (died with pending criminal cases)',
       '1 340 years imprisonment',
       'life imprisonment with no possibility of parole for 25 years',
      

In [62]:
# CLASSIFYING CATEGORIES FOR THE PENALTIES

penalty_categories = {'Death Sentence': [],
            'Life Imprisonment': [],
            'Years Imprisonment': [],
            'Hospital Detention': [],
            'Not Sentenced': [],
            'Suicide or Died Before Sentence': []}

In [63]:
# SORTING THE COLUMN TO ASSIGN TO THE CATEGORIES BASED ON KEY WORKDS - ADDING TEXTING FORMATATION AND RETURING NONE TO NULLS

def categorize_sentence(sentence):
    if pd.isna(sentence):
        return None
    s = str(sentence).lower()
    if any(term in s for term in ['death', 'execution', 'hanging', 'electrocution', 'lethal injection', 'gas chamber']):
        return 'Death Sentence'
    elif 'life' in s:
        return 'Life Imprisonment'
    elif 'year' in s or 'years' in s:
        return 'Years Imprisonment'
    elif any(term in s for term in ['hospital', 'psychiatric', 'mental']):
        return 'Hospital Detention'
    elif any(term in s for term in ['never sentenced', 'never tried', 'not guilty by reason of insanity']):
        return 'Not Sentenced'
    elif any(term in s for term in ['died', 'suicide', 'killed himself']):
        return 'Suicide or Died Before Sentence'
    else:
        return 'other'  # TO UNCATEGORIZED

# CRIATING NEW COLUMN TO ASSIGN THE CATEGORIZATION

profile['Criminal Penalty Categories'] = profile['Criminal Penalty'].apply(categorize_sentence)

In [64]:
profile['Criminal Penalty Categories'].value_counts()


Criminal Penalty Categories
Death Sentence                     247
Life Imprisonment                  237
Years Imprisonment                  66
Hospital Detention                   9
other                                7
Suicide or Died Before Sentence      6
Not Sentenced                        5
Name: count, dtype: int64

In [65]:
profile['Conviction'].unique()

array([nan, 'Vehicle Theft, Murder, Rape', 'Child Se Abuse, Murder',
       'Aggravated Murder Attempted Aggravated Murder', 'Murder',
       'Murder Rape',
       'Murder With Aggravating Circumstances, Seual Abuse, Seual Assault',
       'Aggravated First Degree Murder, Tampering With Evidence, Solicitation',
       'Murder Rape Robbery', 'Murder, Rape, Robbery',
       'Murder With Aggravating Circumstances',
       'Iowa, Sodomy, Illinois, Murder Indecent Liberties With A Child, Deviate Seual Assault',
       'Murder And Rape', 'Murder Attempted Murder Assault',
       'Doubly Qualified Homicide, Abuse Against Dead Bodies, Concealment Of Corpses',
       'Second-Degree Murder ×', 'Manslaughter, Arson',
       'First-Degree Murder', 'Never Convicted', 'First Degree Murder',
       'Murder, Attempted Molestation, Breaking And Entering',
       'Murder, Rape', 'First Degree Murder Necrophilia',
       'First Degree Murder With Special Circumstances Robbery Sodomy',
       'First-Degre

In [66]:
# DICTIONARY TO FIX TYPOS LISTED FROM THE CONVICTION COLUMN

typo_fixes = {
    r"\bSeual\b": "Sexual",
    r"\bSeually\b": "Sexually",
    r"\bSeualy\b": "Sexually",  # corrected spelling
    r"\bEposure\b": "Exposure",
    r"\bEploitation\b": "Exploitation",
    r"\bEtortion\b": "Extortion",
    r"\bOfense\b": "Offense",
    r"\bOfences\b": "Offenses",
    r"\bOf Murder\b": "Murder",
    r"\bOf Rape\b": "Rape",
    r"\bOf Kidnapping\b": "Kidnapping",
    r"\bTeas\b": "Texas",
    r"\bIntoication\b": "Intoxication",
    r"\bDesecrating\b": "Desecration",
    r"\bAggravated Seual Assault\b": "Aggravated Sexual Assault",
    r"\bSeual Battery\b": "Sexual Battery",
    r"\bFelonious Seual\b": "Felonious Sexual",
    r"\bCriminal Seual Conduct\b": "Criminal Sexual Conduct",
    r"\bArmed Robbery Count\b": "Armed Robbery",
    r"\bSeual Abuse\b": "Sexual Abuse",
    r"\bSeual Assault\b": "Sexual Assault",
    r"\bSeual Offense\b": "Sexual Offense",
    r"\bSeual Penetration\b": "Sexual Penetration",
    r"\bSeual Crimes\b": "Sexual Crimes",
    r"\bSeual Conduct\b": "Sexual Conduct"}

In [67]:
# FUNCTION TO CLEAN TYPOS

def fix_typos_simple(text_list, typo_dict):
    def clean(text):
        if not isinstance(text, str):
            return text
        for typo, correction in typo_dict.items():
            text = re.sub(typo, correction, text, flags=re.IGNORECASE)
        return text.strip()
    
    return [clean(t) for t in text_list]

# CALLING CLEANING FUNCTION

profile['Conviction'] = fix_typos_simple(profile['Conviction'], typo_fixes)



In [68]:
# CLEANING TITLES AND REORDERING COLUMNS

columnstoorder = ['Name', 
                  'Nicknames', 
                  'Age', 
                  'Born Date', 
                  'Born Location', 
                  'born country',
                  'Continent',
                  'Died Date', 
                  'Died Location',
                  'Date Apprehended', 
                  'Apprehended Age', 
                  'Conviction', 
                  'Criminal Penalty',
                  'Criminal Penalty Categories',                 
                  'Zodiac Sign']


profile = profile[columnstoorder]
profile.columns = profile.columns.str.title()

In [69]:
profile.head()

,Name,Nicknames,Age,Born Date,Born Location,Born Country,Continent,Died Date,Died Location,Date Apprehended,Apprehended Age,Conviction,Criminal Penalty,Criminal Penalty Categories,Zodiac Sign
0,Luis Garavito,"The Beast, Goofy, Conflict, Bonifacio Morera L...",66.0,1957-01-25,"Gnova, Quindo, Colombia",Colombia,South America,2023-10-12,"Valledupar, Colombia",1999-04-22,42,NaN,NaN,None,Aquarius
1,Pedro López,The Monster of the Andes,NaN,1948-10-08,"Venadillo, Colombia",Colombia,South America,NaN,NaN,1980-03-09,31,"Vehicle Theft, Murder, Rape","7 years imprisonment in Colombia, extended to ...",Years Imprisonment,Libra
2,Javed Iqbal,Kukri,NaN,NaT,"Lahore, Punjab, Pakistan",Pakistan,Asia,NaN,"Lahore, Punjab, Pakistan",1999-12-30,<NA>,"Child Se Abuse, Murder",death,Death Sentence,None
3,Mikhail Popkov,"The Werewolf, The Angarsk Maniac, The Wednesda...",NaN,1964-03-07,"Norilsk, Krasnoyarsk Krai, Russian SFSR, USSR",USSR,Europe,NaN,NaN,2012-06-23,48,Aggravated Murder Attempted Aggravated Murder,life imprisonment,Life Imprisonment,Pisces
4,Daniel Camargo Barbosa,"The Sadist of El Charquito, The Monster of the...",64.0,1930-01-22,"Anolaima, Cundinamarca, Colombia",Colombia,South America,1994-11-13,"Guayaquil, Ecuador",1974-05-03,44,NaN,"25 years imprisonment, 16 years imprisonment",Years Imprisonment,Aquarius


In [70]:
# SAVING CLEAN FILE
profile.to_csv('../Clean Files/Profile/1V-profile_killers.csv', index=False)

### EDA VICTIMS:


In [71]:
victims = pd.read_csv('../Clean Files/Victims/0V-serial_killers_by_victims.csv')

In [72]:
# IDENTIFYING ALL UNIQUE VALUES TO HELP WITH DICTIONARY CREATION

list(victims['Country'].unique())

['Colombia, Ecuador, Venezuela',
 'Colombia, Peru, Ecuador',
 'Pakistan',
 'Russia',
 'Colombia, Ecuador, Brazil',
 'Brazil',
 'India',
 'Iran',
 'China',
 'Afghanistan',
 'United States',
 'Belgian Congo, Tanganyika',
 'Soviet Union',
 'Soviet Union, Ukraine',
 'Indonesia',
 'South Africa',
 'Morocco',
 'Kingdom of Romania, Yugoslavia, Hungary',
 'Mexico',
 'Ottoman Empire, Iraq, Iran',
 'Egypt',
 'Germany',
 'Colombia',
 'Zambia',
 'Swaziland',
 'Greece',
 'Canada',
 'United Kingdom',
 'Austria-Hungary',
 'France, Germany, Switzerland',
 'Ecuador',
 'United States, Canada',
 'Soviet Union, Russia, Tajikistan',
 'Romania',
 'South Korea',
 'Costa Rica',
 'Soviet Union, Russia',
 'Latvia',
 'Turkey',
 'France',
 'Sri Lanka',
 'Peru',
 'Ukraine',
 'Italy',
 'France, Belgium',
 'Spain',
 'United States, Mexico',
 'Argentina',
 'Poland',
 'Chile',
 'West Germany',
 'Tunisia',
 'Venezuela',
 'Rwanda',
 'Thailand, Nepal, India, Malaysia',
 'Yemen',
 'Japan',
 'Austria, United States, Czecho

In [73]:
# DICTIONARY CREATION CATEGORIZING COUNTRIES BY CONTINENT, INCLUDING MULTIPLE CONTINENTS

country_to_continent = {
    'Colombia, Ecuador, Venezuela': 'South America',
    'Colombia, Peru, Ecuador': 'South America',
    'Pakistan': 'Asia',
    'Russia': 'Europe',
    'Colombia, Ecuador, Brazil': 'South America',
    'Brazil': 'South America',
    'India': 'Asia',
    'Iran': 'Asia',
    'China': 'Asia',
    'Afghanistan': 'Asia',
    'United States': 'North America',
    'Belgian Congo, Tanganyika': 'Multiple continents',
    'Soviet Union': 'Multiple continents',
    'Soviet Union, Ukraine': 'Multiple continents',
    'Indonesia': 'Asia',
    'South Africa': 'Africa',
    'Morocco': 'Africa',
    'Kingdom of Romania, Yugoslavia, Hungary': 'Europe',
    'Mexico': 'North America',
    'Ottoman Empire, Iraq, Iran': 'Multiple continents',
    'Egypt': 'Africa',
    'Germany': 'Europe',
    'Colombia': 'South America',
    'Zambia': 'Africa',
    'Swaziland': 'Africa',
    'Greece': 'Europe',
    'Canada': 'North America',
    'United Kingdom': 'Europe',
    'Austria-Hungary': 'Multiple continents',
    'France, Germany, Switzerland': 'Europe',
    'Ecuador': 'South America',
    'United States, Canada': 'North America',
    'Soviet Union, Russia, Tajikistan': 'Multiple continents',
    'Romania': 'Europe',
    'South Korea': 'Asia',
    'Costa Rica': 'North America',
    'Soviet Union, Russia': 'Multiple continents',
    'Latvia': 'Europe',
    'Turkey': 'Asia',
    'France': 'Europe',
    'Sri Lanka': 'Asia',
    'Peru': 'South America',
    'Ukraine': 'Europe',
    'Italy': 'Europe',
    'France, Belgium': 'Europe',
    'Spain': 'Europe',
    'United States, Mexico': 'North America',
    'Argentina': 'South America',
    'Poland': 'Europe',
    'Chile': 'South America',
    'West Germany': 'Europe',
    'Tunisia': 'Africa',
    'Venezuela': 'South America',
    'Rwanda': 'Africa',
    'Thailand, Nepal, India, Malaysia': 'Asia',
    'Yemen': 'Asia',
    'Japan': 'Asia',
    'Austria, United States, Czechoslovakia': 'Europe',
    'Belgium': 'Europe',
    'Bangladesh': 'Asia',
    'Czechoslovakia': 'Europe',
    'Soviet Union, Russia, Ukraine': 'Multiple continents',
    'Kenya': 'Africa',
    'German Empire, Germany': 'Europe',
    'Denmark': 'Europe',
    'Ecuador, Spain': 'South America',
    'Czechoslovakia, Germany, Netherlands': 'Europe',
    'Sweden': 'Europe',
    'China, Japan': 'Asia',
    'Australia': 'Oceania',
    'Spain, France, Italy': 'Europe',
    'Canada, United States': 'North America',
    'Cyprus': 'Asia',
    'Italy, France': 'Europe',
    'East Germany, Germany, France, Spain, Czech Republic, Italy': 'Europe',
    'Belarus': 'Europe',
    'East Germany, Germany': 'Europe',
    'Estonia': 'Europe',
    'United States, Portuguese Angola': 'North America',
    'Switzerland, France, United States, Yugoslavia, Italy': 'Europe',
    'West Germany, United States': 'Europe',
    'Netherlands': 'Europe',
    'SR Croatia': 'Europe',
    'Panama': 'North America',
    'Singapore': 'Asia',
    'Belize': 'North America',
    'SR Slovenia': 'Europe',
    'Namibia': 'Africa',
    'United Kingdom, Ireland, West Germany, Netherlands, France': 'Europe',
    'Austria': 'Europe',
    'Hong Kong': 'Asia',
    'Israel': 'Asia',
    'Allied-occupied Germany, West Germany': 'Multiple continents',
    'Republic of Macedonia': 'Europe',
    'East Germany': 'Europe',
    'SR Slovenia, Serbia and Montenegro, Slovenia': 'Europe',
    'Portugal': 'Europe',
    'Malaysia': 'Asia',
    'United States, Austria, France, United Kingdom': 'North America',
    'Singapore, Malaysia': 'Asia',
    'Kingdom of Yugoslavia': 'Europe',
    'Iraq': 'Asia',
    'Norway': 'Europe',
    'Switzerland': 'Europe',
    'Czech Republic': 'Europe',
    'Imperial State of Iran': 'Multiple continents',
    'Finland': 'Europe',
    'United States, Zimbabwe': 'North America',
    'Hungary': 'Europe',
    'Belgium, France': 'Europe',
    'Guatemala': 'North America',
    'Uzbekistan, Kazakhstan': 'Asia',
    'Italy, West Germany, Netherlands': 'Europe'
}

In [74]:
# APPLYING THE COLUMN CREATION TO ADD CONTINENT

victims['Continent'] = victims['Country'].map(country_to_continent)

In [75]:
# CONFIRMING THE CHANGE AND COUNTING VALUES

victims['Continent'].value_counts()

Continent
North America          325
Europe                 214
Asia                    95
Africa                  39
South America           33
Multiple continents     31
Oceania                 20
Name: count, dtype: int64

In [76]:
# FUNCTION TO CATEGORIZE COUNTRIES AS UNIQUE OR MULTIPLE COUNTRIES:

def categorize_multiple_countries(country_series):
    return country_series.apply(lambda x: "Multiple Countries" if ',' in x else x)

In [77]:
# APPLYING THE CATEGORIES TO CREATE THE NEW COLUMN 'COUNTRY SORTED'

victims['Country sorted'] = categorize_multiple_countries(victims['Country'])

In [78]:
# CONFIRM COLUMN CREATING, AND CHECKING NULLS AND DATA TYPE

victims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757 entries, 0 to 756
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              757 non-null    object 
 1   Country           757 non-null    object 
 2   Start year        738 non-null    float64
 3   End year          740 non-null    float64
 4   Proven victims    757 non-null    float64
 5   Possible victims  757 non-null    float64
 6   Notes             754 non-null    object 
 7   Continent         757 non-null    object 
 8   Country sorted    757 non-null    object 
dtypes: float64(4), object(5)
memory usage: 53.4+ KB


In [79]:
# CONVERTING FLOATS TO INTEGER

victims[['Start year', 'End year', 'Proven victims', 'Possible victims']] = victims[['Start year', 'End year', 'Proven victims', 'Possible victims']].astype('Int64')


In [80]:
# CREATING COLUMN OF MEDIAN YEARS OF THE CRIMES:

victims['Median Year'] = (victims['Start year'] + victims['End year']) / 2

victims['Median Year'] = victims['Median Year'].astype('Int64')


In [81]:
# CREATING COLUMN DECADE AND CONFERTING AS STRING

victims['Decade'] = (victims['Median Year'] // 10) * 10

victims['Decade'] = victims['Decade'].astype('Int64').astype(str) + 's'

victims['Decade'] = victims['Decade'].replace('<NA>', 'Unknown')


In [82]:
# CREATING DICTIONARY AND APPKYING TO CATEGORIZE KILLERS BY GENRE

female_killers = [
    'Marie Fikáčková', 'Ivanova and Olga Tamarin', 'Gwendolyn Graham and Catherine May Wood',
    'Felícitas Sánchez Aguillón', 'Delfina and María de Jesús González', 'Angel Makers of Nagyrév',
    'Lainz Angels of Death', 'Silvia Merez Cult', 'Amelia Sach and Annie Walters', 'Belle Gunness',
    'Aileen Wuornos', 'Dorothea Puente', 'Juana Barraza', 'Jane Toppan', 'Genene Jones',
    'Beverley Allitt', 'Enriqueta Martí', 'Tillie Klimek', 'Mariane Nölle', 'Vickie Dawn Jackson',
    'Joan Vila', 'Linda Hazzard', 'Ann Arber Hospital Killer', 'Elizabeth Wettlaufer', 'Velma Barfield',
    'Lyda Southard', 'Shirley Winters', 'Francisca Ballesteros', 'Valery Asratyan', 'Martha Rendell',
    'Sharon Kinne', 'Leonarda Cianciulli', 'Bertha Gifford', 'Rhonda Belle Martin', 'Reta Mays',
    'Marybeth Tinning', 'Jeanne Weber', 'Marie Alexandrine Becker', 'Antoinette Scieri', 'Kristen Gilbert',
    'Sonya Caleffi', 'Kimberly Clark Saenz', 'Vera Renczi', 'Mariam Soulakiotis', 'Irina Gaidamachuk',
    'Tamara Samsonova', 'Celine Lesage', 'Elisabeth Wiese', 'Anna Marie Hahn', 'Waneta Hoyt',
    'Lucy Letby', 'Frans Hooijmaijers', 'Miyuki Ishikawa', 'Amy Archer-Gilligan', 'Aino Nykopp-Koski',
    'Elfriede Blauensteiner', 'Joanna Dennehy', 'Mary Elizabeth Wilson'
]

gangs = [
    "Raya and Sakina and their husbands", "Beasts of Satan", "Chijon Family", "The Family",
    "Ian Brady and Myra Hindley", "Ray and Faye Copeland", "The Skin Hunters", "The Hernández Brothers Sect",
    "The Ciudad Juárez Rebels", "Tadeusz Grzesik and the Bureaucrats Gang", "The Manson Family cult killings",
    "National Socialist Underground", "Joshi-Abhyankar serial murderers", "Hillside Stranglers",
    "Bloody Benders", "The Cleaners", "Lainz Angels of Death", "Death Angels", "Ripper Crew",
    "Anísio Ferreira de Sousa and followers", "Marcelo de Jesus Silva and his death squad", "Shankill Butchers",
    "Brabant killers", "Gang of Amazons", "Murder Incorporated", "Auto Shankar's Gang", "Mailoni Brothers",
    "Briley Brothers", "David Avendaño Ballina and his followers", "Philadelphia poison ring"
]

couples = [
    "Michael Bear Carson and Suzan Carson", "Paul Bernardo and Karla Homolka", "Raymond Fernandez and Martha Beck",
    "David and Catherine Birnie", "James Gregory Marlow and Cynthia Coffman", "Rudolfo Infante and Anna Villeda",
    "Gerald and Charlene Gallego", "Juan Carlos Hernández and Patricia Martínez", "Fred West and Rosemary West",
    "Adolfo Constanzo and Sara Aldrete", "Ray and Faye Copeland", "Ian Brady and Myra Hindley",
    "Futoshi Matsunaga and Junko Ogata", "Long Zhimin and Yan Shuxia", "Abboud and Khajawa"
]

male_groups = [
    "Joseph and Michael Kallinger", "James Paster and Stephen McCoy", "John Duffy and David Mulcahy",
    "Loren Herzog and Wesley Shermantine", "Lawrence Bittaker and Roy Norris",
    "Artyom Alexandrovich Anoufriev and Nikita Vakhtangovich Lytkin", "David Alan Gore and Fred Waterfield",
    "Hermann Duft and Hans Wilhelm Bassenauer", "Kiss brothers and Zsolt István Pető", "Gert van Rooyen and Joey Haarhof",
    "Johannes van Rooyen and Dumisani Makhubela", "Fa Ziying and Lao Rongzhi", "Christopher Worrell and James Miller",
    "Dale Hausner and Samuel Dieteman", "Edgar Álvarez Cruz and Francisco Granados", "Song Jinghua and Yan Jinguang",
    "Yevgeny Nagorny and Sergei Stavitsky", "Wolfgang Abel and Marco Furlan", "Polatbay Berdaliyev and Abduseit Ormanov",
    "Shen Changyin and Shen Changpin", "Leonard Lake and Charles Ng", "John Justin Bunting, Robert Joe Wagner and James Spyridon Vlassakis",
    "José Miculax Bux and Mariano Macú Miculax", "Vladyslav Volkovic and Volodymyr Kondratenko",
    "John Allen Muhammad and Lee Boyd Malvo", "Wang Zongfang and Wang Zongwei", "Viktor Sayenko and Igor Suprunyuk",
    "Dean Corll, David Owen Brooks and Elmer Wayne Henley", "Bian Kuang, Fu Xinyuan and Luo Lianshun",
    "Hassan Orangi and Abbas Ali Zarifian", "Peng Miaoji, Ding Yunjia and Su Xiaoping"
]

unidentified = [
    "Monster of the Mangones", "Cape Town Prostitute Killer", "El Psicópata",
    "Atlanta Ripper", "Cleveland Torso Murderer", "Rainbow Maniac", "Paraquat murders",
    "Long Island serial killer", "New Bedford Highway", "Edgecombe County Serial Killer",
    "Axeman of New Orleans", "Forest City Killer", "Łódź Gay Murderer", "The Doodler", "Bible John","Monster of Florence"
]

# FUNCTION TO APPLY:

def classify_killer(name):
    if name in female_killers:
        return 'Female'
    elif name in gangs:
        return 'Gang'
    elif name in couples:
        return 'Couple'
    elif name in male_groups:
        return 'Male Group'
    elif name in unidentified:
        return 'Unidentified'
    else:
        return 'Male'

# APPLYING THE FUNCTION TO THE DATA FRAME

victims['Killer Type'] =  victims['Name'].apply(classify_killer)

In [83]:
#CRIATING POSSIBLE NON PROVEN VICTIMS, WHICH IS THE DIFFERENCE FROM POSSIBLE VICTMS AND PROVEN

victims['Unproven Possible Victims'] = victims['Possible victims'] - victims['Proven victims']


In [84]:
# RENAMING POSSIBLE VICTIMS TO 'Total victims (Unproven + Proven)' TO BE MORE DISCRITIVE

victims.rename(columns={'Possible victims': 'Total victims (Unproven + Proven)'}, inplace=True)


In [85]:
# CREATING COLUNN 'YEARS ACTIVE' TO UNDERSTAND THE PERIOD OF ACTIVITY FROM KILLERS/ CRIMES

victims['Years Active'] = victims['End year'] - victims['Start year']


In [86]:
# REORDERING COLUMNS BY SIMILARITY

toreorder = ['Name',
             'Killer Type', 
             'Country', 
             'Country sorted' , 
             'Continent',
             'Start year', 
             'End year', 
             'Median Year', 
             'Years Active',
             'Decade', 
             'Total victims (Unproven + Proven)',
             'Unproven Possible Victims',
             'Proven victims',
             'Notes']

victims = victims[toreorder]


In [87]:
# ALL COLUMNS NAMES TO TITLE

victims.columns = victims.columns.str.title()


In [88]:
# CONFIRMING ALL CHANGES

victims.head(2)

,Name,Killer Type,Country,Country Sorted,Continent,Start Year,End Year,Median Year,Years Active,Decade,Total Victims (Unproven + Proven),Unproven Possible Victims,Proven Victims,Notes
0,Luis Garavito,Male,"Colombia, Ecuador, Venezuela",Multiple Countries,South America,1992,1999,1995,7,1990s,247,54,193,"Child-murderer, torture-killer, and rapist kno..."
1,Pedro López,Male,"Colombia, Peru, Ecuador",Multiple Countries,South America,1969,1980,1974,11,1970s,300,190,110,"Child-murderer and rapist, known as ""The Monst..."


In [89]:
victims.to_csv('../Clean Files/Victims/1V-serial_killers_by_victims.csv', index=False)

### CREATING TOP 15 KILLERS FILE

In [90]:
top15 = pd.read_csv('../Clean Files/Victims/1V-serial_killers_by_victims.csv')

In [91]:
# SORTING THE TOP 15 BASED ON 'POSSIBLE VICTIMS' , THIS COLUMN HAS BEEN IDENTIFYIED AS A POSSIBLE TOTAL VICTIMS LATER

top15 = top15.sort_values(by='Total Victims (Unproven + Proven)', ascending=False,).head(15).copy()

In [92]:
# CREATION OF DICTIONARY INCLUDING URL WITH KILLER PHOTO TO INCLUDE AS A COLUMN LATER

imagenes = {
    'Murder Incorporated': 'https://www.legendsofamerica.com/wp-content/uploads/2017/08/Murderinc.jpg',
    'The Skin Hunters': 'https://static.wikia.nocookie.net/skdb/images/2/2c/Andrzej_Nowocie%C5%84.jpg/revision/latest?cb=20230707162353',
    'Charles Cullen': 'https://media-cldnry.s-nbcnews.com/image/upload/t_fit-1500w,f_auto,q_auto:best/msnbc/Components/Photos/060319/060319_cullen_vsml_8a.jpg',
    'Amelia Sach and Annie Walters': 'https://pm1.aminoapps.com/6044/e102b3c5568cd78a3fb4e16d3d4d4f032f526a8b_00.jpg',
    'Pedro López': 'https://s3.amazonaws.com/arc-wordpress-client-uploads/infobae-wp/wp-content/uploads/2018/11/14100939/Pedro-Alonso-Lopez-el-Monstruo-de-los-Andes-3.jpg',
    'Angel Makers of Nagyrév': 'https://www.economist.com/cdn-cgi/image/width=1424,quality=80,format=auto/content-assets/images/20230318_CUP001.jpg',
    'Niels Högel': 'https://www.lavanguardia.com/files/content_image_mobile_filter/uploads/2022/04/21/6261a4bb37515.jpeg',
    'Abul Djabar': 'https://alpha-assets.tadevel-cdn.com/663d766925bc4c6ee7e00f81/720.webp',
    'Frans Hooijmaijers': 'https://thumbor.pijper.io/5EwhMNHH-OViNv253yb-ej8DMuY=/728x0/https%3A%2F%2Fcdn.pijper.io%2F2024%2F11%2Fq6DZBqjmCkIjDH1731685697.png',
    'Harold Shipman': 'https://m.media-amazon.com/images/S/pv-target-images/43d413170f9c3edaac3eceefaedc067b210f7cd44f5566140bceeb13ac977637.jpg',
    'Luis Garavito': 'https://upload.wikimedia.org/wikipedia/commons/7/7b/Luis_Alfredo_Garavito_Cubillos.jpg',
    'Kermit Gosnell': 'https://media.philly.com/images/20140213_dn_0n0wr3al.jpg',
    'Daniel Camargo Barbosa': 'https://upload.wikimedia.org/wikipedia/commons/e/ef/Daniel_Camargo_Barbosa_1958_Mugshot.png',
    'Mariam Soulakiotis': 'https://images.findagrave.com/photos/2024/251/245406222_a2b9c415-c479-410f-afd9-71f5b42a883a.jpeg',
    'Richard Kuklinski': 'https://swashvillage.org/storage/img/images/richard-the-iceman-kuklinski-biography_2.jpg'
}


In [93]:
# CREATING COLUMN WITH PHOTO OF THE KILLERS

top15['Image'] = top15['Name'].map(imagenes)

In [94]:
top15.head()

,Name,Killer Type,Country,Country Sorted,Continent,Start Year,End Year,Median Year,Years Active,Decade,Total Victims (Unproven + Proven),Unproven Possible Victims,Proven Victims,Notes,Image
675,Murder Incorporated,Gang,United States,United States,North America,1920.0,NaN,NaN,NaN,<NA>s,1000,600,400,Murder Incorporated or Murder Inc. for short w...,https://www.legendsofamerica.com/wp-content/up...
737,The Skin Hunters,Gang,Poland,Poland,Europe,NaN,2002.0,NaN,NaN,<NA>s,620,615,5,Hospital crew that killed patients in order to...,https://static.wikia.nocookie.net/skdb/images/...
631,Charles Cullen,Male,United States,United States,North America,1988.0,2003.0,1995.0,15.0,1990s,400,371,29,Nurse in New Jersey and Pennsylvania who murde...,https://media-cldnry.s-nbcnews.com/image/uploa...
756,Amelia Sach and Annie Walters,Female,United Kingdom,United Kingdom,Europe,1900.0,1903.0,1901.0,3.0,1900s,300,299,1,"Sach and Walters, known as the ""Finchley baby ...",https://pm1.aminoapps.com/6044/e102b3c5568cd78...
1,Pedro López,Male,"Colombia, Peru, Ecuador",Multiple Countries,South America,1969.0,1980.0,1974.0,11.0,1970s,300,190,110,"Child-murderer and rapist, known as ""The Monst...",https://s3.amazonaws.com/arc-wordpress-client-...


In [95]:
# OBSERVED SOME OF THE COLUMNS WENT BACK TO FLOAT, THEN HAVING THEM AS INT

top15[['Start Year', 'End Year', 'Median Year', 'Years Active']] = top15[['Start Year', 'End Year', 'Median Year', 'Years Active']].astype('Int64')


In [96]:
#CONFIRMING DATA FORMAT

top15.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 675 to 490
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Name                               15 non-null     object
 1   Killer Type                        15 non-null     object
 2   Country                            15 non-null     object
 3   Country Sorted                     15 non-null     object
 4   Continent                          15 non-null     object
 5   Start Year                         12 non-null     Int64 
 6   End Year                           14 non-null     Int64 
 7   Median Year                        11 non-null     Int64 
 8   Years Active                       11 non-null     Int64 
 9   Decade                             15 non-null     object
 10  Total Victims (Unproven + Proven)  15 non-null     int64 
 11  Unproven Possible Victims          15 non-null     int64 
 12  Proven Victi

In [97]:
top15.to_csv('../Clean Files/Victims/top15_killers_by_victims.csv', index=False)